検証用・雑記用のnotebook

# モジュールのインポート

In [9]:
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm

from modules.constants import LocalDirs, HorseResultsCols
from modules import preparing
from modules import preprocessing
from modules import training
from modules import simulation
from modules import policies
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# horse_idマスタ検証

In [10]:
%autoreload

In [6]:
pd.read_csv('data/master/horse_id.csv')

,horse_id,encoded_id
0,2018106541,0
1,2018103708,1
2,2018103805,2
3,2018101171,3
4,2018103832,4
...,...,...
11516,2019101360,11516
11517,2019101500,11517
11518,2019104312,11518
11519,2018102219,11519


In [4]:
max(pd.read_csv('data/master/horse_id.csv')['encoded_id'])

11520

In [4]:
sample = pd.DataFrame(columns=['horse_id', 'encoded_id'])
sample

,horse_id,encoded_id


In [5]:
len(sample)

0

In [6]:
max(sample['encoded_id'])

ValueError: max() arg is an empty sequence

In [7]:
#rawデータファイルパスの取得
files_results = glob.glob(os.path.join(LocalDirs.RAW_RESULTS_DIR, '*.pickle'))
files_race_info = glob.glob(os.path.join(LocalDirs.RAW_RACE_INFO_DIR, '*.pickle'))
files_return = glob.glob(os.path.join(LocalDirs.RAW_RETURN_DIR, '*.pickle'))
files_horse_results = glob.glob(os.path.join(LocalDirs.RAW_HORSE_RESULTS_DIR, '*.pickle'))
files_peds = glob.glob(os.path.join(LocalDirs.RAW_PEDS_DIR, '*.pickle'))

In [8]:
#前処理
results_processor = preprocessing.ResultsProcessor(files_results)
race_info_processor = preprocessing.RaceInfoProcessor(files_race_info)
return_processor = preprocessing.ReturnProcessor(files_return)
horse_results_processor = preprocessing.HorseResultsProcessor(files_horse_results)
peds_processor = preprocessing.PedsProcessor(files_peds)

In [11]:
### 馬の過去成績を集計しつつ、前処理の済みの全てのテーブルをマージする処理 ###

#ターゲットエンコーディング時に「馬の成績」として扱う項目
TARGET_COLS = [HorseResultsCols.RANK, HorseResultsCols.PRIZE, HorseResultsCols.RANK_DIFF, 
               'first_corner', 'final_corner',
               'first_to_rank', 'first_to_final','final_to_rank']
#horse_id列と共に、ターゲットエンコーディングの対象にする列
GROUP_COLS = ['course_len', 'race_type', HorseResultsCols.PLACE]

data_merger = preprocessing.DataMerger(
        results_processor,
        race_info_processor,
        horse_results_processor,
        peds_processor,
        target_cols=TARGET_COLS,
        group_cols=GROUP_COLS
)
data_merger.merge() #処理実行

separating horse results by date


  0%|          | 0/106 [00:00<?, ?it/s]

merging horse_results


  0%|          | 0/106 [00:00<?, ?it/s]

In [12]:
# 元々のマスタをhorse_id_2.csvなどに変更して、初期時の挙動をチェック
feature_enginnering = preprocessing.FeatureEngineering(data_merger)\
    .add_interval()\
        .dumminize_ground_state()\
            .dumminize_race_type()\
                .dumminize_sex()\
                    .dumminize_weather()\
                        .encode_horse_id()\
                            .encode_jockey_id()\
                                .dumminize_kaisai()

正しくマスタができているのでOK。ファイルを元に戻す。